<a href="https://colab.research.google.com/github/steimel60/ML/blob/main/DeepLearning/TrainingAModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.vision.all import *

     |████████████████████████████████| 719 kB 5.1 MB/s 
     |████████████████████████████████| 4.2 MB 64.4 MB/s 
     |████████████████████████████████| 346 kB 38.4 MB/s 
     |████████████████████████████████| 1.2 MB 54.6 MB/s 
     |████████████████████████████████| 197 kB 69.4 MB/s 
     |████████████████████████████████| 59 kB 9.4 MB/s 
     |████████████████████████████████| 1.1 MB 63.6 MB/s 
     |████████████████████████████████| 86 kB 8.2 MB/s 
     |████████████████████████████████| 140 kB 71.1 MB/s 
     |████████████████████████████████| 212 kB 77.5 MB/s 
     |████████████████████████████████| 86 kB 6.7 MB/s 
     |████████████████████████████████| 596 kB 59.3 MB/s 
     |████████████████████████████████| 127 kB 70.7 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
     |████████████████████████████████| 144 kB 68.5 MB/s 
     |████████████████████████████████| 271 kB 6.3 MB/s 
     |████████████████████████████████| 6.6 MB 30.0 MB/s 
ERROR: pip's dependency 

In [3]:
#Load a subset of the Image Net database
path = untar_data(URLs.IMAGENETTE)

In [4]:
dblock = DataBlock(
    blocks = (ImageBlock(), CategoryBlock()),
    get_items=get_image_files,
    get_y=parent_label,
    item_tfms=Resize(460),
    batch_tfms=aug_transforms(size=224, min_scale=0.75)
)
dls = dblock.dataloaders(path, bs=64)

In [5]:
#Base line training run
model = xresnet50()
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(5, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.622628,4.081925,0.318521,03:05
1,1.235935,2.212081,0.406647,03:13
2,0.957124,0.976631,0.682226,03:14
3,0.745823,0.643310,0.800224,03:13
4,0.617888,0.563034,0.823376,03:13


When training a model it helps to Normalize the input data

In [6]:
#Define a func that gets normalized data loaders
def get_dls(bs, size):
  dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                     get_items=get_image_files,
                     get_y=parent_label,
                     item_tfms=Resize(460),
                     batch_tfms=[*aug_transforms(size=size, min_scale=0.75), Normalize.from_stats(*imagenet_stats)])
  return dblock.dataloaders(path, bs=bs)
dls = get_dls(64,224)
x,y=dls.one_batch()
x.mean(dim=[0,2,3]), x.std(dim=[0,2,3])

(TensorImage([-0.0981, -0.0431,  0.0960], device='cuda:0'),
 TensorImage([1.2040, 1.2081, 1.3363], device='cuda:0'))

Progressive Resizing (start with small images work to big) helps speed up training for larger images and also is a for of data augmentation - leading to better generalization

In [7]:
#Start small
dls = get_dls(128,128)
learn = Learner(dls, xresnet50(), loss_func=CrossEntropyLossFlat(),metrics=accuracy)
learn.fit_one_cycle(4, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.859342,1.995422,0.456684,02:17
1,1.291904,1.523966,0.548917,02:17
2,0.948959,0.922987,0.711725,02:17
3,0.737039,0.634304,0.801718,02:17


In [8]:
#Replace DataLoaders inside learner and fine-tune with larger images
learn.dls = get_dls(64, 224)
learn.fine_tune(5, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.825953,0.828410,0.736744,03:15


epoch,train_loss,valid_loss,accuracy,time
0,0.646897,0.648605,0.787528,03:16
1,0.652248,0.600548,0.806572,03:16
2,0.566100,0.525756,0.835325,03:15
3,0.491676,0.462569,0.848021,03:16
4,0.415891,0.423530,0.862957,03:15


Test Time Augmentation: When using random cropping fastai autmatically uses center cropping. Most options have their pros and cons. It can be useful to use the averages of all augmentations on our validation set to get a better idea of our models accuracy.

In [9]:
#TTA gives boost in performance with no additional training
preds, targs=learn.tta()
accuracy(preds, targs).item()

0.8752800822257996

Mixup: Another form of data augmentation useful when you don't have much data or a pretrained model. Basically, mixes to images together and takes the average of the label predictions as final target depending how much of each image is used.